In [1]:
version = 'v20250521'

## creat pixels

In [2]:
# import rasterio
# from rasterio.mask import mask
# import os
# os.environ['USE_PYGEOS'] = '0'
# import geopandas as gpd
# from shapely.geometry import Point
# import numpy as np
# import pandas as pd
# from skmap.misc import find_files, GoogleSheet, ttprint

# rtif = 'http://192.168.1.30:8333/ai4sh-landmasked/air.temp/clm_air.temp_era5.copernicus_mx_1km_200..200cm_2019.04.01..2019.04.30_eumap_epsg3035_v0.1.tif'

# ttprint('de')
# polys = gpd.read_file("./material/nuts_de_2021.gpkg")         
# with rasterio.open(rtif) as src:
#     # out_image → ndarray (bands, rows, cols)
#     # out_transform → affine transform for the masked array
#     out_image, out_transform = mask(src, polys.geometry, crop=True)
#     out_image = out_image[0]                    # assume one-band raster
#     crs = src.crs                               # keep raster CRS

# height, width = out_image.shape
# rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
# xs, ys = rasterio.transform.xy(out_transform, rows, cols, offset='center')
# df = pd.DataFrame({
#     "x": np.array(xs).flatten(),
#     "y": np.array(ys).flatten(),
#     "value": out_image.flatten()
# })

# de = gpd.GeoDataFrame(
#         df,
#         geometry=[Point(x, y) for x, y in zip(df.x, df.y)],
#         crs=crs
# )

# ttprint('es')
# polys = gpd.read_file("./material/nuts_es_2021.gpkg")         
# with rasterio.open(rtif) as src:
#     out_image, out_transform = mask(src, polys.geometry, crop=True)
#     out_image = out_image[0]                    # assume one-band raster
#     crs = src.crs                               # keep raster CRS

# height, width = out_image.shape
# rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
# xs, ys = rasterio.transform.xy(out_transform, rows, cols, offset='center')
# df = pd.DataFrame({
#     "x": np.array(xs).flatten(),
#     "y": np.array(ys).flatten(),
#     "value": out_image.flatten()
# })
# es = gpd.GeoDataFrame(
#         df,
#         geometry=[Point(x, y) for x, y in zip(df.x, df.y)],
#         crs=crs
# )

# ttprint('finish')

# de['nuts0'] = 'DE'
# es['nuts0'] = 'ES'

# agg_pixel = pd.concat([de,es])
# agg_pixel = agg_pixel.drop(columns=['geometry','value'])
# agg_pixel.to_parquet('./material/pixel_agg.pq')

## read back in the overlaid version

In [3]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import copy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold,cross_val_predict
from skmap.misc import find_files, GoogleSheet, ttprint
import joblib
import numpy as np

prop = 'soc'
tgt = 'soc_log1p'

# df = pd.read_parquet('./pixel_agg.pq')

covs = pd.read_csv(f'./feature_selected_soc_{version}.txt', header=None)[0].tolist() 

df = pd.read_parquet(f'./agg.pixel.tmp_overlaid_{version}.pq')
print(df.shape)
dff = df.dropna(subset=covs,how='any')
print(dff.shape)
# dff = dff.groupby('id').filter(lambda x: len(x) >= 3).reset_index(drop=True)
# print(dff.shape)

(7678990, 79)
(2335589, 79)


## cast the models

In [4]:
# read in model
model = joblib.load(f'./model_rf.soc_ccc_{version}.joblib')
model.n_jobs = 90

# use all the points to train model
train = pd.read_parquet(f'./soc.topsoil_organized_{version}.pq')
train[tgt] = np.log1p(train[prop])
train = train.dropna(subset=covs+[tgt],how='any').reset_index(drop=True)

# fit
model.fit(train[covs], train[tgt])

# cast the model
import copy
from trees_rf import cast_tree_rf, cast_node_rf, pad_leaf_outputs_to_array
import warnings
warnings.filterwarnings("ignore", message="X has feature names, but")
ttprint('casting the model')
model_copy = copy.deepcopy(model)
modeln = cast_node_rf(model_copy, train[covs], train[tgt])
ttprint('finish casting')

[14:18:17] casting the model
[14:18:40] finish casting


## start prediction

In [5]:
# prediction
ttprint('start prediction')
y_pred = model.predict(dff[covs])
y_pred = np.expm1(y_pred)
dff[f'pred'] = y_pred
ttprint('finish prediction')

[14:18:40] start prediction
[14:18:41] finish prediction


/tmp/ipykernel_65218/1822879946.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff[f'pred'] = y_pred


In [6]:
# do the predictions in chunck
chunk_size = 10000
num_chunks = int(np.ceil(len(dff) / chunk_size))

pred_std_chunks = []

ttprint(f'In total {num_chunks} chunks')

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(dff))
    ttprint(f'chunk {i + 1}/{num_chunks} ({start}:{end})--------')

    chunk = dff.iloc[start:end]

    ttprint('start predicting and padding')
    node_preds_chunk = modeln.predict(chunk[covs])
    nodes_chunk = pad_leaf_outputs_to_array(node_preds_chunk, pad_value=np.nan)
    nodes_chunk = np.expm1(nodes_chunk)
    
    ttprint('start calculating std')
    std_chunk = np.nanstd(nodes_chunk.T, axis=0)
    pred_std_chunks.append(std_chunk)

ttprint(f'finish all-------------------------------')

dff['pred_std'] = np.concatenate(pred_std_chunks)

dff.to_parquet(f'pixel_agg.predicted_all.{version}.pq')

[14:18:41] In total 234 chunks
[14:18:41] chunk 1/234 (0:10000)--------
[14:18:41] start predicting and padding
[14:19:34] start calculating std
[14:19:34] chunk 2/234 (10000:20000)--------
[14:19:34] start predicting and padding
[14:20:27] start calculating std
[14:20:27] chunk 3/234 (20000:30000)--------
[14:20:27] start predicting and padding
[14:21:19] start calculating std
[14:21:20] chunk 4/234 (30000:40000)--------
[14:21:20] start predicting and padding
[14:22:12] start calculating std
[14:22:13] chunk 5/234 (40000:50000)--------
[14:22:13] start predicting and padding
[14:23:05] start calculating std
[14:23:05] chunk 6/234 (50000:60000)--------
[14:23:05] start predicting and padding
[14:23:58] start calculating std
[14:23:58] chunk 7/234 (60000:70000)--------
[14:23:58] start predicting and padding
[14:24:51] start calculating std
[14:24:51] chunk 8/234 (70000:80000)--------
[14:24:51] start predicting and padding
[14:25:44] start calculating std
[14:25:45] chunk 9/234 (80000

/tmp/ipykernel_65218/720670535.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['pred_std'] = np.concatenate(pred_std_chunks)


OSError: Cannot save file into a non-existent directory: 'material'

In [18]:
dff = dff.rename(columns={'ndvi_glad.landast.ard2.seasconv.m.yearly_p50_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20231127':'ndvi'})

out = (
    dff
    .pivot_table(
        index=['nuts0', 'lon', 'lat', 'x', 'y'],
        columns='time',
        values=['pred', 'pred_std', 'ndvi'],
        aggfunc='first'       
    )
    .dropna(axis=0)           
    .reset_index()
)

out.columns = [
    f'{c[0]}_{c[1]}' if c[1] else c[0]   
    for c in out.columns.to_flat_index()
]

static_cols = ['nuts0', 'lon', 'lat', 'x', 'y']
value_cols  = ['pred_2009', 'pred_2018', 'pred_std_2009', 'pred_std_2018', 'ndvi_2009', 'ndvi_2018']
out = out[static_cols + value_cols]

out.head()

,nuts0,lon,lat,x,y,pred_2009,pred_2018,pred_std_2009,pred_std_2018,ndvi_2009,ndvi_2018
0,DE,4031500.0,2684510.0,4031500.0,2684510.0,50.580191,51.958039,55.865364,58.576836,221.0,221.0
1,DE,4031500.0,2685510.0,4031500.0,2685510.0,61.634346,50.056710,63.864868,60.460979,223.0,226.0
2,DE,4031500.0,2686510.0,4031500.0,2686510.0,46.685048,56.671120,37.056778,56.921967,225.0,231.0
3,DE,4031500.0,2687510.0,4031500.0,2687510.0,45.154006,43.176903,39.046261,38.734066,220.0,223.0
4,DE,4031500.0,2688510.0,4031500.0,2688510.0,26.473726,24.404207,36.337608,26.744663,199.0,194.0


In [20]:
import numpy as np
out['change'] = out['pred_2018'] - out['pred_2009']
out['noise'] = np.sqrt(out['pred_std_2009']**2 + out['pred_std_2018']**2)
out['signal'] = out['change'].abs()
out['SNR'] = out['signal']/out['noise']
out['ndvi_mean'] = (out['ndvi_2009'] + out['ndvi_2018'])/2
out['ndvi_mean'] = (out['ndvi_mean'] - 125)/125
out = out.drop(columns=['x','y'])

In [22]:
out.to_parquet('agg_pixel_change.pq')

## save as geotiff

In [25]:
geometry = [Point(xy) for xy in zip(out['lon'], out['lat'])]
gdf = gpd.GeoDataFrame(out, geometry=geometry, crs="EPSG:3035")
de = gpd.read_file("nuts_de_2021.gpkg")         
es = gpd.read_file("nuts_es_2021.gpkg")         
poly = gpd.GeoDataFrame(pd.concat([es, de], ignore_index=True), crs="EPSG:3035")
gdf = gpd.sjoin(gdf, poly, how="inner", predicate="within")

In [27]:
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

value_col = "SNR"        
points_gdf = gdf.dropna(subset=[value_col]) 

# ------------------------------------------------------------------
# 2.  DERIVE GRID RESOLUTION & EXTENT ------------------------------
# ------------------------------------------------------------------

x_res = 1000    # ≈ pixel width  (e.g. 1000 m)
y_res = 1000           # ≈ pixel height (e.g. 1000 m)

minx, miny, maxx, maxy = points_gdf.total_bounds

# number of pixels in each direction (+1 so last column/row is included)
width  = int(round((maxx - minx) / x_res)) + 1
height = int(round((maxy - miny) / y_res)) + 1

# Affine transform (upper-left corner is half a pixel north-west of first point)
transform = from_origin(minx - x_res / 2,   # west-most edge
                        maxy + y_res / 2,   # north-most edge
                        x_res,
                        y_res)

# ------------------------------------------------------------------
# 3.  RASTERIZE POINT VALUES ---------------------------------------
# ------------------------------------------------------------------
shapes = ((geom, val) for geom, val in
          zip(points_gdf.geometry, points_gdf[value_col]))

raster = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=np.nan,             # background
    dtype="float32"          # change if your data need another type
)

# ------------------------------------------------------------------
# 4.  WRITE GeoTIFF -----------------------------------------------
# ------------------------------------------------------------------
meta = {
    "driver":  "GTiff",
    "height":  height,
    "width":   width,
    "count":   1,
    "dtype":   "float32",
    "crs":     points_gdf.crs,   # keep original CRS
    "transform": transform,
    "nodata":  np.nan
}

with rasterio.open(f"{value_col}.tif", "w", **meta) as dst:
    dst.write(raster, 1)

print("GeoTIFF written to predictions.tif")

GeoTIFF written to predictions.tif
